In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import mlflow
from mrmr import mrmr_classif
import os
import matplotlib.pyplot as plt
import preprocesamiento_datos

In [12]:
# Establecer la URI de la base de datos SQLite y configuración de MLflow
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("regresion_lineal")

<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1713193978222, experiment_id='3', last_update_time=1713193978222, lifecycle_stage='active', name='regresion_lineal', tags={}>

# Selección con MRMR

In [14]:
# Iniciar el registro de la ejecución en MLflow
with mlflow.start_run(run_name="MRMR Feature Selection"):
    import numpy as np
    # Obtener los datos preprocesados
    X_train, X_val, X_test, y_train, y_val, y_test = preprocesamiento_datos.preprocesamiento(False,[], False)
    
    # Definir la función para calcular el MRMR
    def calculate_mrmr_features(X, y, K):
        selected_features = mrmr_classif(X=X, y=y, K=K)
        return selected_features
    
    # Definir el rango de características que deseamos considerar
    feature_range = range(1, len(X_train.columns) + 1)
    
    # Listas para almacenar las métricas
    train_maes = []
    test_maes = []
    num_features_list = []
    
    # Variables para almacenar las 17 características y su MAE correspondiente
    selected_features_17 = None
    mae_17 = None
    
    # Iterar sobre todas las combinaciones de características
    for num_features in feature_range:
        # Calcular las características seleccionadas usando MRMR
        selected_features = calculate_mrmr_features(X_train, y_train, num_features)
        
        # Seleccionar las columnas correspondientes en X_train y X_test
        X_train_selected = X_train[selected_features]
        X_test_selected = X_test[selected_features]
        
        # Entrenar el modelo de regresión lineal
        model = LinearRegression()
        model.fit(X_train_selected, y_train)
    
        # Realizar predicciones en el conjunto de entrenamiento y de prueba
        y_pred_train = model.predict(X_train_selected)
        y_pred_test = model.predict(X_test_selected)
        
        # Calcular el error absoluto medio (MAE) para evaluar el modelo en ambos conjuntos
        train_mae = mean_absolute_error(y_train, y_pred_train)
        test_mae = mean_absolute_error(y_test, y_pred_test)
        
        # Almacenar las métricas y el número de características
        train_maes.append(train_mae)
        test_maes.append(test_mae)
        num_features_list.append(num_features)
        
        # Imprimir el MAE en cada iteración
        print(f" {num_features} - Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
        
        # Registrar el MAE en MLflow
        mlflow.log_metric(f"train_mae_{num_features}", train_mae)
        mlflow.log_metric(f"test_mae_{num_features}", test_mae)
        print(feature_range[np.argmin(test_maes)])
        print(min(test_maes))
    # Graficar los resultados
    plt.plot(num_features_list, train_maes, label='Train MAE')
    plt.plot(num_features_list, test_maes, label='Test MAE')
    plt.xlabel('Number of Features')
    plt.ylabel('Mean Absolute Error (MAE)')
    plt.title('MAE vs. Number of Features')
    plt.legend()
    plt.show()

Archivo Datos_la_liga_preparados_entrenamiento.parquet guardado en: ../Downloads\Datos_la_liga_preparados_entrenamiento.parquet


KeyboardInterrupt: 

In [10]:
import mlflow
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt

# Iniciar el registro de la ejecución en MLflow
with mlflow.start_run(run_name="KBest Feature Selection"):

    # Obtener los datos preprocesados
    X_train, X_val, X_test, y_train, y_val, y_test = preprocesamiento_datos.preprocesamiento(False,[], False)
    
    # Definir el rango de características que deseamos considerar
    feature_range = range(1, 100)
    
    # Listas para almacenar las métricas
    train_maes = []
    test_maes = []
    
    # Iterar sobre todas las combinaciones de características
    for num_features in feature_range:
        # Seleccionar las mejores k características
        selector = SelectKBest(score_func=f_regression, k=num_features)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)
        
        # Entrenar un modelo de regresión lineal con las características seleccionadas
        model = LinearRegression()
        model.fit(X_train_selected, y_train)
    
        # Realizar predicciones en el conjunto de prueba
        y_pred_train = model.predict(X_train_selected)
        y_pred_test = model.predict(X_test_selected)
        
        # Calcular el error absoluto medio (MAE) para evaluar el modelo en ambos conjuntos
        train_mae = mean_absolute_error(y_train, y_pred_train)
        test_mae = mean_absolute_error(y_test, y_pred_test)
        
        # Almacenar las métricas
        train_maes.append(train_mae)
        test_maes.append(test_mae)
        
        # Registrar los MAE en MLflow para cada iteración
        mlflow.log_metric(f"train_mae_{num_features}", train_mae)
        mlflow.log_metric(f"test_mae_{num_features}", test_mae)
        
        # Verificar si se han encontrado las 9 características
        if num_features == 9:
            # Obtener las características seleccionadas
            selected_features_9 = np.array(X_train.columns)[selector.get_support()]
            print(f"Selected features with {num_features} features:", selected_features_9)
            # Registrar las características seleccionadas en MLflow
            mlflow.log_param("selected_features_9", selected_features_9)
        
        if num_features == 12:
            # Obtener las características seleccionadas
            selected_features_9 = np.array(X_train.columns)[selector.get_support()]
            print(f"Selected features with {num_features} features:", selected_features_9)
            # Registrar las características seleccionadas en MLflow
            mlflow.log_param("selected_features_9", selected_features_9)
        # Verificar si se han encontrado las 17 características
        if num_features == 17:
            # Obtener las características seleccionadas
            selected_features_17 = np.array(X_train.columns)[selector.get_support()]
            print(f"Selected features with {num_features} features:", selected_features_17)
            # Registrar las características seleccionadas en MLflow
            mlflow.log_param("selected_features_17", selected_features_17)
            
        if num_features == 20:
            # Obtener las características seleccionadas
            selected_features_20 = np.array(X_train.columns)[selector.get_support()]
            print(f"Selected features with {num_features} features:", selected_features_20)
            # Registrar las características seleccionadas en MLflow
            mlflow.log_param("selected_features_20", selected_features_20)
    
    # Encontrar el número de características que produce el menor error absoluto medio en el conjunto de prueba
    best_num_features = feature_range[np.argmin(test_maes)]
    print("Best number of features:", best_num_features)
    
    # Graficar los resultados
    plt.plot(feature_range, train_maes, label='Train MAE')
    plt.plot(feature_range, test_maes, label='Test MAE')
    plt.scatter(17, test_maes[16], color='green', label='Test 17')
    plt.scatter(9, test_maes[8], color='green', label='Test 9')
    plt.scatter(12, test_maes[11], color='green', label='Test 12')
    plt.scatter(best_num_features, min(test_maes), color='red', label='Best Test MAE (20)')
    plt.xlabel('Number of Features')
    plt.ylabel('Mean Absolute Error (MAE)')
    plt.title('MAE vs. Number of Features')
    plt.legend()
    plt.show()
    image_path = "grafica_selección_variables.png"
    plt.savefig(image_path)
    mlflow.log_artifact(image_path, artifact_path="grafica seleccion variables")

Archivo Datos_la_liga_preparados_entrenamiento.parquet guardado en: ../Downloads\Datos_la_liga_preparados_entrenamiento.parquet
Selected features with 9 features: ['Mins' 'AvgP' 'Titularidades' 'Equipo_pos' '1_año_anterior'
 '2_año_anterior' '3_año_anterior' '4_año_anterior' '5_año_anterior']
Selected features with 12 features: ['Mins' 'Assists' 'MotM' 'AvgP' 'Titularidades' 'Equipo_pos'
 '1_año_anterior' '2_año_anterior' '3_año_anterior' '4_año_anterior'
 '5_año_anterior' 'Equipo_Real Madrid']


MlflowException: INVALID_PARAMETER_VALUE: Changing param values is not allowed. Param with key='selected_features_9' was already logged with value='['Mins' 'AvgP' 'Titularidades' 'Equipo_pos' '1_año_anterior'
 '2_año_anterior' '3_año_anterior' '4_año_anterior' '5_año_anterior']' for run ID='b4a8c5042eed4bbf8eae9edf80d35461'. Attempted logging new value '['Mins' 'Assists' 'MotM' 'AvgP' 'Titularidades' 'Equipo_pos'
 '1_año_anterior' '2_año_anterior' '3_año_anterior' '4_año_anterior'
 '5_año_anterior' 'Equipo_Real Madrid']'.

The cause of this error is typically due to repeated calls
to an individual run_id event logging.

Incorrect Example:
---------------------------------------
with mlflow.start_run():
    mlflow.log_param("depth", 3)
    mlflow.log_param("depth", 5)
---------------------------------------

Which will throw an MlflowException for overwriting a
logged parameter.

Correct Example:
---------------------------------------
with mlflow.start_run():
    with mlflow.start_run(